# WebSurferAgent

AutoGen provides a proof-of-concept WebSurferAgent that can command a simple text-based browser (similar to [Lynx](https://en.wikipedia.org/wiki/Lynx_(web_browser))) to search the web, visit pages, navigate within pages, download files, etc. The browsing is stateful, meaning that browsing history, viewport state, and other details are maintained throughout the conversation. 

This work was largely inspired by OpenAI's [WebGPT](https://openai.com/research/webgpt) project from December 2021. 

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install AutoGen with the optional `websurfer` dependencies:
```bash
pip install "pyautogen[websurfer]"
```

In [1]:
#%pip install --quiet "pyautogen[websurfer]"

ERROR: Exception:
Traceback (most recent call last):
  File "/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.11/http/client.py", line 473, in read
    s = self.fp.r

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The WebSurferAgent uses a combination of models. GPT-4 and GPT-3.5-turbo-16 are recommended.

Your json config should look something like the following:
```json
[
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key here>"
    },
    {
        "model": "gpt-3.5-turbo-16k",
        "api_key": "<your OpenAI API key here>"
    }
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.


In [2]:
import os

import os
config_list = [{"api_type":"groq","model":"llama3-70b-8192","api_key":os.environ.get("GROQ_API_KEY"),}]
llm_config={"config_list" : config_list}


In [4]:
import autogen  # noqa: E402

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config,#autogen.config_list_from_json(
        #"OAI_CONFIG_LIST",
        #filter_dict={"model": ["gpt-4", "gpt-4-0613", "gpt-4-32k", "gpt-4-32k-0613", "gpt-4-1106-preview"]},
    #),
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config,#autogen.config_list_from_json(
        #"OAI_CONFIG_LIST",
        #filter_dict={"model": ["gpt-3.5-turbo-1106", "gpt-3.5-turbo-16k-0613", "gpt-3.5-turbo-16k"]},
    #),
    "temperature": 0,
}

## Configure Bing

For WebSurferAgent to be reasonably useful, it needs to be able to search the web -- and that means it needs a Bing API key. 
You can read more about how to get an API on the [Bing Web Search API](https://www.microsoft.com/en-us/bing/apis/bing-web-search-api) page.

Once you have your key, either set it as the `BING_API_KEY` system environment variable, or simply input your key below.


In [5]:
import os  # noqa: E402

bing_api_key = os.environ["BING_API_KEY"]

KeyError: 'BING_API_KEY'

### Construct Agents

We now create out WebSurferAgent, and a UserProxyAgent to surf the web. 

In [4]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent  # noqa: E402

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": bing_api_key},
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

### Example 1: Search, summarize
- Search for information about Microsoft AutoGen
- Summarize the results
- Visit the Getting Started Docs page

In [5]:
task1 = """
Search the web for information about Microsoft AutoGen
"""

user_proxy.initiate_chat(web_surfer, message=task1)

user_proxy (to web_surfer):


Search the web for information about Microsoft AutoGen


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: bing: Microsoft AutoGen
Title: Microsoft AutoGen - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'Microsoft AutoGen' found 10 results:

## Web Results
1. [AutoGen: Enabling next-generation large language model applications](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/)
AutoGen is a Python package that simplifies the orchestration, optimization, and automation of large language model applications. It enables customizable and conversable agents that integrate with humans, tools, and other agents to solve tasks using GPT-4 and other advanced LLMs. Learn how to use AutoGen for code-based question answering, supply-c

In [6]:
task2 = "Summarize these results"
user_proxy.initiate_chat(web_surfer, message=task2, clear_history=False)

user_proxy (to web_surfer):

Summarize these results

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION summarize_page...
web_surfer (to user_proxy):

AutoGen is a Python package and framework developed by Microsoft that simplifies the orchestration, optimization, and automation of large language model (LLM) applications. It enables the development of customizable and conversable agents that can solve tasks using advanced LLMs like GPT-4. AutoGen supports various conversation patterns, enhanced LLM inference, and seamless integration with humans, tools, and other agents. It offers a high-level abstraction for building diverse and enhanced LLM workflows and provides a collection of working systems for different domains and complexities. AutoGen is open-source and supports natural language and code-based conversation patterns for applications such as question answering, coding, mathematics, and more

In [7]:
task3 = "Click the 'Getting Started' result"
user_proxy.initiate_chat(web_surfer, message=task3, clear_history=False)

user_proxy (to web_surfer):

Click the 'Getting Started' result

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION navigational_web_search...
web_surfer (to user_proxy):

Address: https://microsoft.github.io/autogen/docs/Getting-Started/
Title: Getting Started | AutoGen
Viewport position: Showing page 1 of 2.
Getting Started | AutoGen

[Skip to main content](#)[![AutoGen](/autogen/img/ag.svg)![AutoGen](/autogen/img/ag.svg)**AutoGen**](/autogen/)[Docs](/autogen/docs/Getting-Started)[SDK](/autogen/docs/reference/agentchat/conversable_agent)[Blog](/autogen/blog)[FAQ](/autogen/docs/FAQ)[Examples](/autogen/docs/Examples)Resources* [Ecosystem](/autogen/docs/Ecosystem)
* [Gallery](/autogen/docs/Gallery)
[GitHub](https://github.com/microsoft/autogen)🌜🌞`ctrl``K`* [Getting Started](/autogen/docs/Getting-Started)
* [Installation](/autogen/docs/Installation)
* [Use Cases](#)
* [Contributing](/autogen/docs/Con

### Example 2: Navigational search, scroll, answer questions
- Search for Microsoft's wikipedia page, then naviagate to it
- Scroll down
- Answer questions about the content

In [8]:
task4 = """Find Microsoft's Wikipedia page."""
user_proxy.initiate_chat(web_surfer, message=task4, clear_history=False)

user_proxy (to web_surfer):

Find Microsoft's Wikipedia page.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION navigational_web_search...
web_surfer (to user_proxy):

Address: https://en.wikipedia.org/wiki/Microsoft
Title: Microsoft - Wikipedia
Viewport position: Showing page 1 of 64.
# Microsoft

American multinational technology corporation

Microsoft Corporation| [A square divided into four sub-squares, colored red-orange, green, yellow and blue (clockwise), with the company name appearing to its right](/wiki/File:Microsoft_logo_(2012).svg) |
| Building 92 on the [Microsoft Redmond campus](/wiki/Microsoft_Redmond_campus "Microsoft Redmond campus") |
| Type | [Public](/wiki/Public_company "Public company") |
| [Traded as](/wiki/Ticker_symbol "Ticker symbol") | * [Nasdaq](/wiki/Nasdaq "Nasdaq"): [MSFT](https://www.nasdaq.com/market-activity/stocks/msft)
* [Nasdaq-100](/wiki/Nasdaq-100 "Nasdaq-1

In [9]:
task5 = """Scroll down."""
user_proxy.initiate_chat(web_surfer, message=task5, clear_history=False)

user_proxy (to web_surfer):

Scroll down.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION page_down...
web_surfer (to user_proxy):

Address: https://en.wikipedia.org/wiki/Microsoft
Title: Microsoft - Wikipedia
Viewport position: Showing page 2 of 64.
(finance)") | Increase US$206.2 billion (2023) |
| Number of employees | 238,000 (2023) |
| [Divisions](/wiki/Division_(business) "Division (business)") | 
* [Microsoft Engineering Groups](/wiki/Microsoft_engineering_groups "Microsoft engineering groups")
* [Microsoft Digital Crimes Unit](/wiki/Microsoft_Digital_Crimes_Unit "Microsoft Digital Crimes Unit")
* [Microsoft Press](/wiki/Microsoft_Press "Microsoft Press")
* [Microsoft Japan](/wiki/Microsoft_Japan "Microsoft Japan")
* [Microsoft Gaming](/wiki/Microsoft_Gaming "Microsoft Gaming")

 |
| [Subsidiaries](/wiki/Subsidiary "Subsidiary") | 
* [GitHub](/wiki/GitHub "GitHub")
* [LinkedIn](/wiki/Lin

In [10]:
task6 = """Where was the first office location, and when did they move to Redmond?"""
user_proxy.initiate_chat(web_surfer, message=task6, clear_history=False)

user_proxy (to web_surfer):

Where was the first office location, and when did they move to Redmond?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION answer_from_page...
web_surfer (to user_proxy):

Microsoft's first office location was in Albuquerque, New Mexico, where it was founded on April 4, 1975. However, Microsoft later moved its headquarters to Redmond, Washington in January 1979. Since then, Redmond has been the main office location for Microsoft.

--------------------------------------------------------------------------------
